In [67]:
import numpy as np
import pretty_midi as pm
import io
import pickle

In [23]:
info = np.loadtxt("./groove/info.csv", delimiter=",", dtype="str")[1:]

In [64]:
tones = [36, 38, 40, 37, 48, 50, 45, 47, 43, 58, 46, 26, 42, 22, 44, 49, 55, 57, 52, 51, 59, 53]
tone_map = {}
tones.sort()
print(tones)

[22, 26, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 57, 58, 59]


In [66]:
for i, tone in enumerate(tones):
    tone_map[tone] = i
print(tone_map)

{22: 0, 26: 1, 36: 2, 37: 3, 38: 4, 40: 5, 42: 6, 43: 7, 44: 8, 45: 9, 46: 10, 47: 11, 48: 12, 49: 13, 50: 14, 51: 15, 52: 16, 53: 17, 55: 18, 57: 19, 58: 20, 59: 21}


In [97]:
def convert_to_sequence(midi_data):
    time_resolution = midi_data.tick_to_time(1)
    
    duration = midi_data.get_end_time()
    total_ticks = int(duration / time_resolution) + 1
    
    note_sequence = np.zeros((total_ticks, len(tones)))
    
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            tick_start = int(note.start / time_resolution)
            tick_end = int(note.end / time_resolution)
            
            note_sequence[tick_start:tick_end, tone_map[note.pitch]] = note.velocity
    
    return note_sequence

In [98]:
test = []

for meta in info:
    split = meta[10]
    midi_path = meta[7]
    midi_data = pm.PrettyMIDI("./groove/" + midi_path)
    type = meta[5]
    if (type == "fill" and split == "train"):
        test.append(convert_to_sequence(midi_data))
        

In [99]:
test = np.array(test)
with open('trainfill.pickle', 'wb') as f:
    pickle.dump(test, f)

/var/folders/7l/846g42h920x3crq90t2kn9sh0000gn/T/ipykernel_99076/1565148663.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test = np.array(test)


In [94]:
with open('trainfill.pickle', 'rb') as f:
    load = pickle.load(f)

(2785, 150)
